In [ ]:
!pip install schedule python-dotenv


In [ ]:
import os

# temporary, only for this Colab session
os.environ["EMAIL_USER"] = "tayabajameel14@gmail.com"
os.environ["EMAIL_PASS"] = "ugsgzdjhfbqytcph"   # the 16-char App Password you created


In [ ]:
import imaplib, email, smtplib
from email.mime.text import MIMEText
import os, time

USER = os.getenv("EMAIL_USER")
PASS = os.getenv("EMAIL_PASS")

def check_inbox():
    print("📥 Checking inbox...")
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(USER, PASS)
    mail.select("inbox")

    result, data = mail.search(None, '(UNSEEN)')
    mail_ids = data[0].split()

    if not mail_ids:
        print("No new emails.\n")
    else:
        for num in mail_ids:
            result, msg_data = mail.fetch(num, "(RFC822)")
            raw_email = msg_data[0][1]
            msg = email.message_from_bytes(raw_email)
            subject = msg["subject"]
            sender = msg["from"]
            print(f"New email → {subject} from {sender}")

            if "invoice" in subject.lower():
                send_reply(sender, subject)

    mail.logout()

def send_reply(to_addr, subject):
    msg = MIMEText("Thank you for your email Tayaba will reach you soon. This is an automated reply 😊")
    msg["Subject"] = "Re: " + subject
    msg["From"] = USER
    msg["To"] = to_addr

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(USER, PASS)
        server.send_message(msg)
    print(f"✅ Auto-reply sent to {to_addr}\n")

# Run once
check_inbox()


📥 Checking inbox...


AttributeError: 'NoneType' object has no attribute 'replace'

In [ ]:
import imaplib
import email
from email.header import decode_header
import pandas as pd

# --- 1️⃣ Gmail Credentials (App Password required) ---
EMAIL_USER = "tayabajameel14@gmail.com"     # your Gmail address
EMAIL_PASS = "ugsgzdjhfbqytcph"             # your 16-character app password

# --- 2️⃣ Connect to Gmail IMAP ---
imap = imaplib.IMAP4_SSL("imap.gmail.com")
imap.login(EMAIL_USER, EMAIL_PASS)
imap.select("inbox")

# --- 3️⃣ Fetch unread emails ---
status, messages = imap.search(None, "UNSEEN")
email_ids = messages[0].split()
print(f"📥 Found {len(email_ids)} unread emails.\n")

# --- 4️⃣ Define categorization rules (for now) ---
def categorize_email(subject, sender):
    subject_lower = subject.lower()
    sender_lower = sender.lower()

    if any(word in subject_lower for word in ["security", "alert", "verification", "login"]):
        return "Security"
    elif any(word in sender_lower for word in ["amazon", "newsletter", "promo", "marketing", "offers"]):
        return "Promotions"
    else:
        return "General"

# --- 5️⃣ Store emails in a DataFrame ---
data = []

for mail_id in email_ids:
    status, msg_data = imap.fetch(mail_id, "(RFC822)")
    msg = email.message_from_bytes(msg_data[0][1])

    # Decode email subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding or "utf-8", errors="ignore")

    # Get sender
    sender = msg.get("From")

    # Categorize
    category = categorize_email(subject, sender)

    # Append to list
    data.append({
        "Sender": sender,
        "Subject": subject,
        "Category": category
    })

# Convert to DataFrame
email_df = pd.DataFrame(data)

# --- 6️⃣ Close connection ---
imap.close()
imap.logout()

# Show first few emails
email_df.head()


📥 Found 3 unread emails.



,Sender,Subject,Category
0,Google <no-reply@accounts.google.com>,Security alert,Security
1,Google <no-reply@google.com>,"✅ Tayaba, finish setting up your Redmi 10 2022...",General
2,Google <no-reply@accounts.google.com>,Security alert,Security


In [ ]:
!pip install pandas scikit-learn matplotlib seaborn
!pip install transformers  # optional if you want summaries


In [ ]:
import imaplib

# 1️⃣ Connect securely
imap = imaplib.IMAP4_SSL("imap.gmail.com")

# 2️⃣ Login
imap.login("tayabajameel14@gmail.com", "ugsgzdjhfbqytcph")

# 3️⃣ Select mailbox
imap.select("inbox")  # You must select a mailbox before searching

# 4️⃣ Search emails
status, messages = imap.search(None, "ALL")
email_ids = messages[0].split()
print(f"Found {len(email_ids)} emails")



Found 33 emails


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Example training data (expandable)
train_subjects = [
    "Your Amazon order has shipped",
    "Login alert for your Gmail",
    "Meeting at 3 PM",
    "Special marketing offers just for you",
    "Password verification required"
]
train_labels = ["Promotions", "Security", "General", "Promotions", "Security"]

# TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_subjects)

# Train classifier
clf = LogisticRegression()
clf.fit(X_train, train_labels)

# Predict categories for fetched emails
email_df['Category_ML'] = clf.predict(vectorizer.transform(email_df['Subject']))

# Show first few emails with ML category
email_df.head()


,Sender,Subject,Category,Category_ML
0,Google <no-reply@accounts.google.com>,Security alert,Security,Security
1,Google <no-reply@google.com>,"✅ Tayaba, finish setting up your Redmi 10 2022...",General,Security
2,Google <no-reply@accounts.google.com>,Security alert,Security,Security


In [ ]:
!pip install -q transformers


In [ ]:

import imaplib
import email
from email.header import decode_header
import pandas as pd
from transformers import pipeline


In [ ]:
EMAIL_USER = "tayabajameel14@gmail.com"
EMAIL_PASS = "ugsgzdjhfbqytcph"

# Connect to Gmail IMAP
imap = imaplib.IMAP4_SSL("imap.gmail.com")
imap.login(EMAIL_USER, EMAIL_PASS)
imap.select("inbox")

# Fetch ALL emails
status, messages = imap.search(None, "ALL")
email_ids = messages[0].split()
print(f"📥 Total emails found: {len(email_ids)}")

rows = []


📥 Total emails found: 33


In [ ]:
for mail_id in email_ids:
    status, msg_data = imap.fetch(mail_id, "(RFC822)")

    msg = email.message_from_bytes(msg_data[0][1])

    # Decode email subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding or "utf-8", errors="ignore")

    # Sender
    from_ = msg.get("From")

    rows.append({
        "Sender": from_,
        "Subject": subject
    })

imap.close()
imap.logout()

# Create DataFrame
email_df = pd.DataFrame(rows)
email_df.head()


,Sender,Subject
0,no-reply@signup.aws,Verify your email address
1,Amazon Web Services <no-reply@amazonaws.com>,Amazon Web Services
2,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier
3,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier
4,Amazon Web Services <aws-marketing-email-repli...,Tell us about your AWS signup experience


In [ ]:
# Load zero-shot classifier
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli"
)

# Categories you want the AI to classify into
labels = ["Promotions", "Security", "General", "Meetings", "Finance", "Updates"]

# Apply model to each email subject
email_df["Category_ML"] = email_df["Subject"].apply(
    lambda s: classifier(s, labels)["labels"][0]
)

email_df.head()


Device set to use cpu


,Sender,Subject,Category_ML
0,no-reply@signup.aws,Verify your email address,Security
1,Amazon Web Services <no-reply@amazonaws.com>,Amazon Web Services,Updates
2,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Promotions
3,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Promotions
4,Amazon Web Services <aws-marketing-email-repli...,Tell us about your AWS signup experience,Updates


In [ ]:
email_df.to_csv("email_report_ml.csv", index=False)
email_df.head()


,Sender,Subject,Category_ML
0,no-reply@signup.aws,Verify your email address,Security
1,Amazon Web Services <no-reply@amazonaws.com>,Amazon Web Services,Updates
2,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Promotions
3,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Promotions
4,Amazon Web Services <aws-marketing-email-repli...,Tell us about your AWS signup experience,Updates


In [7]:
# ======================================
# 1️⃣ Install & Import Libraries
# ======================================
!pip install transformers==4.40.0 sentencepiece

import imaplib
import email
from email.header import decode_header
import pandas as pd
from transformers import pipeline

# ======================================
# 2️⃣ Connect to Gmail
# ======================================
imap = imaplib.IMAP4_SSL("imap.gmail.com")
imap.login("tayabajameel14@gmail.com", "ugsgzdjhfbqytcph")
imap.select("inbox")

# ======================================
# 3️⃣ Fetch all email IDs
# ======================================
status, messages = imap.search(None, "ALL")
email_ids = messages[0].split()

# ======================================
# 4️⃣ Helper to extract body
# ======================================
def get_email_body(msg):
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                try:
                    return part.get_payload(decode=True).decode("utf-8", errors="ignore")
                except:
                    return ""
    else:
        try:
            return msg.get_payload(decode=True).decode("utf-8", errors="ignore")
        except:
            return ""
    return ""

# ======================================
# 5️⃣ Build rows
# ======================================
rows = []
for mail_id in email_ids[:20]:  # limit to 20 emails for speed
    status, msg_data = imap.fetch(mail_id, "(RFC822)")
    msg = email.message_from_bytes(msg_data[0][1])

    # Decode subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding or "utf-8", errors="ignore")

    # Sender
    sender = msg.get("From")

    # Body
    body = get_email_body(msg)

    rows.append({
        "Sender": sender,
        "Subject": subject,
        "Body": body
    })

imap.close()
imap.logout()

# ======================================
# 6️⃣ Create DataFrame
# ======================================
email_df = pd.DataFrame(rows)
email_df.head()

# ======================================
# 7️⃣ Load Transformers Pipelines
# ======================================
# Zero-shot classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Auto-reply generator
reply_generator = pipeline("text-generation", model="gpt2", temperature=0.7)

# Categories
labels = ["Promotions", "Security", "General", "Meetings", "Finance", "Updates"]

# ======================================
# 8️⃣ Classify Emails
# ======================================
email_df["Category_ML"] = email_df["Subject"].apply(lambda s: classifier(s, labels)["labels"][0])

# ======================================
# 9️⃣ Summarize Emails
# ======================================
def summarize_email(text):
    if not text or len(text) < 50:
        return text
    try:
        return summarizer(text[:1024])[0]["summary_text"]
    except:
        return text

email_df["Summary"] = email_df["Body"].apply(summarize_email)

# ======================================
# 🔟 Auto-Reply
# ======================================
def generate_reply(row):
    prompts = {
        "Finance": "Write a short professional reply about a finance-related email:",
        "Security": "Write a reply addressing a security issue:",
        "Promotions": "Write a reply thanking them for the promotion information:",
        "Meetings": "Write a reply confirming the meeting:",
        "Updates": "Write a reply acknowledging the update received:",
        "General": "Write a polite general email reply:"
    }

    prompt = prompts.get(row["Category_ML"], "Write a polite reply email:\n")
    prompt += f"Email: {row['Summary']}\nReply:"

    # Generate short, effective reply using max_new_tokens
    generated = reply_generator(
        prompt,
        max_new_tokens=50,  # <-- ensures short replies and avoids max_length error
        do_sample=True
    )[0]["generated_text"]

    # Remove the prompt part from the generated text
    reply_text = generated.replace(prompt, "").strip()
    return reply_text

# Apply function
email_df["AutoReply"] = email_df.apply(generate_reply, axis=1)

# ======================================
# 1️⃣1️⃣ Final Output
# ======================================
email_df.head()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generatio

,Sender,Subject,Body,Category_ML,Summary,AutoReply
0,no-reply@signup.aws,Verify your email address,Verify your email address\r\n\r\nGreetings fro...,Security,Amazon Web Services will never email you and a...,Ask Amazon to delete your account.\nWhat's in ...
1,Amazon Web Services <no-reply@amazonaws.com>,Amazon Web Services,"<html xmlns=""http://www.w3.org/1999/xhtml""><bo...",Updates,Email Body and Email Header are included in th...,Please confirm you're happy with the update.
2,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Start using services and applications in the A...,Promotions,You're only a few steps away from completing y...,Your account has successfully been activated a...
3,Amazon Web Services <aws-marketing-email-repli...,Activate 12 months of AWS Free Tier,Start using services and applications in the A...,Promotions,You're only a few steps away from completing y...,You're only a few steps away from completing y...
4,Amazon Web Services <aws-marketing-email-repli...,Tell us about your AWS signup experience,Your feedback helps us improve\r\n\r\n<https:/...,Updates,We noticed you recently tried signing up for a...,You have signed up for an Amazon Web Services ...


In [3]:
# Save DataFrame to CSV
email_df.to_csv("email_replies.csv", index=False)

# Check the file
!ls -lh email_replies.csv


-rw-r--r-- 1 root root 115K Nov 26 07:04 email_replies.csv


In [4]:
from google.colab import files
files.download("email_replies.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>